In [611]:
# 加载配置
from importlib import reload
import solver.bid_simulation_data #  import DataGenerator
import simulator.bid_simulate #  import DataGenerator
import config # 配置文件
import config_ct_hot
import simulator.simulator
    
reload(simulator.simulator)
reload(config)
reload(config_ct_hot)
reload(simulator.bid_simulate)
reload(solver.bid_simulation_data)

<module 'solver.bid_simulation_data' from '/root/workspace/ad-strategy/bidding/solver/bid_simulation_data.py'>

In [612]:
job = simulator.bid_simulate.BidSimulator(config_ct_hot, dg.flow_ratio)

In [586]:
# 直接通过内存加载数据（可能会内存超限）
scene = "ct_hot"
date = 20251015
history_data_file = "./data/20251012_20251015.csv"
charge_data_file = "./data/20251012_20251014_charge_budget.csv"
simulation_data_file = "./data/20251015_ct_hot_request_data.csv"
dg = solver.bid_simulation_data.DataGenerator(
    history_data_file, 
    charge_data_file, 
    simulation_data_file, 
    scene,
    date
)

In [608]:
# 获取计划数据
s = dg.simulation_data
s["tcharge"] = s["cpa"] * s["cv"]
s["charge"] = s["price"]
s = s.groupby("plan_id").agg({"tcharge": "sum", "charge": "sum", "cv": "sum"}).reset_index()

# 非新进成本出价计划
pdata = dg.history_data
pdata = pdata[(pdata.bid_type == 1.0) & (pdata.ds == 20251014)].groupby("plan_id").sum().reset_index()
psdata = pd.merge(pdata, s, on="plan_id")

In [609]:
budgets = {}
base_results = {}
new_results = {}

In [ ]:
for plan_id in psdata.plan_id.to_list():
    if plan_id in base_results.keys():
        continue
    print(plan_id)
    plan_data = dg.generate(plan_id)
    base_result, new_result = job.simulate(plan_data)
    base_results[plan_id] = base_result
    new_results[plan_id] = new_result
    budgets[plan_id] = plan_data.budget * dg.simulation_ratio * plan_data.scene_ratio

102212838


In [603]:
plan_id = 102014847
plan_data = dg.generate(plan_id)
budget = plan_data.budget * dg.simulation_ratio * plan_data.scene_ratio

In [ ]:
base_result, new_result = job.simulate(plan_data, True)

In [ ]:
job.summary(base_result, budget)

In [ ]:
job.summary(new_result, budget)